## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-14-05-56-52 +0000,nypost,Michigan woman faces jail after pleading guilt...,https://nypost.com/2026/02/14/us-news/michigan...
1,2026-02-14-05-48-49 +0000,nyt,"Department of Homeland Security Shuts Down, Th...",https://www.nytimes.com/2026/02/13/us/politics...
2,2026-02-14-05-36-09 +0000,nypost,Lindsey Graham’s message to Europeans ‘nervous...,https://nypost.com/2026/02/14/us-news/lindsey-...
3,2026-02-14-05-25-21 +0000,nyt,Casey Wasserman Will Sell Entertainment Agency...,https://www.nytimes.com/2026/02/13/us/casey-wa...
4,2026-02-14-05-17-27 +0000,bbc,"'Canadians are with you,' says PM at Tumbler R...",https://www.bbc.com/news/articles/cde4kdr2z6do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2503/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,44
86,ice,19
34,epstein,19
79,new,15
348,court,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2026-02-13-22-19-13 +0000,latimes,Newsom tells world leaders Trump's retreat on ...,https://www.latimes.com/california/story/2026-...,80
342,2026-02-13-10-00-16 +0000,wapo,Services cut. Staff laid off. New Trump cuts h...,https://www.washingtonpost.com/education/2026/...,79
137,2026-02-13-21-44-46 +0000,nypost,Ted Lieu makes wild claim about Trump and Epst...,https://nypost.com/2026/02/13/us-news/lieu-mak...,76
223,2026-02-13-17-52-55 +0000,nypost,Trump says he plans to visit Venezuela after C...,https://nypost.com/2026/02/13/us-news/trump-sa...,75
209,2026-02-13-18-33-12 +0000,nypost,AOC downplays 2028 talk in Munich — babbles wh...,https://nypost.com/2026/02/13/us-news/aoc-does...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,80,2026-02-13-22-19-13 +0000,latimes,Newsom tells world leaders Trump's retreat on ...,https://www.latimes.com/california/story/2026-...
123,56,2026-02-13-22-13-46 +0000,nypost,Here’s the crude email exchange with Jeffrey E...,https://nypost.com/2026/02/13/business/heres-t...
225,55,2026-02-13-17-39-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
110,47,2026-02-13-22-32-02 +0000,bbc,ICE agents may have lied about shooting of mig...,https://www.bbc.com/news/articles/cy8l9jp1z3yo...
97,44,2026-02-13-22-59-00 +0000,cbc,Labelling Palestine Action a terrorist group w...,https://www.cbc.ca/radio/asithappens/palestine...
156,44,2026-02-13-20-53-59 +0000,bbc,Ex-CNN host Don Lemon pleads not guilty in Min...,https://www.bbc.com/news/articles/cge89r1e158o...
14,43,2026-02-14-04-14-27 +0000,nypost,3 people detained as SWAT team descended on ho...,https://nypost.com/2026/02/13/us-news/swat-tea...
286,40,2026-02-13-13-54-02 +0000,missionlocal,San Francisco teachers end 4-day strike with p...,https://missionlocal.org/2026/02/sf-teachers-s...
293,36,2026-02-13-13-04-33 +0000,nypost,Trans shooter Jesse Van Rootselaar — who kille...,https://nypost.com/2026/02/13/world-news/jesse...
61,33,2026-02-14-00-22-58 +0000,cbc,'Canadians are with you. We will always be wit...,https://www.cbc.ca/news/canada/british-columbi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
